## Machine Learning Lasso
---------------------------------------------------------
### Gas Production (from 1970-2018)
* Goal: Predict the next five years 2019-2023 compare MSE and R-Square to other machine learning models

In [1]:
# Import dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

### Reading, cleaning, and transforming the data with features into a single dataframe

In [2]:
# Read the 'gas production' csv file into a pandas DataFrame
gas_prod = pd.read_csv('../data/clean_data_final/final_clean_data/Gas Production - EJ-YearFixed-Python.csv')
# Replace all the blanks (or NaN) with zero
clean_gas_prod = gas_prod.replace(np.nan,0)
# Read the 'world population' csv file into pandas; adding in 'Population' feature
global_pop = pd.read_csv('../data/clean_data/WorldPopulationbyYear.csv')
# Merge the 'gas production' with 'world population by year'
gas_pop = global_pop.merge(clean_gas_prod, on="Year")
# Renaming columns 
# new_df = df.rename(columns={"A": "a", "B": "c"})
gas_pop = gas_pop.rename(columns={"World": "World Population",
                                  "Total World": "World Exajoules"})
# Adding in 'GDP' as a feature
# Read the 'GDP' csv file into pandas
gdp_percent = pd.read_csv('../data/clean_data_final/final_clean_data/GDP%-YearFixed-Python.csv', encoding = 'ISO-8859-1')
# Replace all the blanks (or NaN) with zero
clean_gdp_percent = gdp_percent.replace(np.nan,0)
filtered_gdp = clean_gdp_percent[['Year', 'United States', 'World']]
filtered_gdp = filtered_gdp.rename(columns={'United States': 'US GDP (%)',
                                            'World': 'World GDP (%)'})
# Merge the latest dataframe with 'filtered GDP'
gas_prod_v1 = filtered_gdp.merge(gas_pop, on="Year")
# Adding in 'Inflation' as a feature
# Read the 'Inflation' csv file into pandas
inflation = pd.read_csv('../data/clean_data_final/final_clean_data/InflationAnnual%-YearFixed-Python.csv', encoding = 'ISO-8859-1')
# Replace all the blanks (or NaN) with zero
clean_inf = inflation.replace(np.nan,0)
filtered_inf = clean_inf[['Year', 'United States', 'World']]
filtered_inf = filtered_inf.rename(columns={'United States': 'US Inflation (Annual %)',
                                            'World': 'World Inflation (Annual %)'})
# Merge the latest dataframe with 'filtered inflation'
gas_prod_v2 = filtered_inf.merge(gas_prod_v1, on="Year")
# Adding in 'Taxes on goods & services' as a feature
# Read the 'Taxes' csv file into pandas
taxes = pd.read_csv('../data/clean_data_final/final_clean_data/Taxes%-YearFixed-Python.csv', encoding = 'ISO-8859-1')
# Replace all the blanks (or NaN) with zero
clean_tax = taxes.replace(np.nan,0)
filtered_tax = clean_tax[['Year', 'United States', 'World']]
filtered_tax = filtered_tax.rename(columns={'United States': 'US Tax (% on Goods & Services)',
                                            'World': 'World Tax (% on Goods & Services)'})
# Merge the latest dataframe with 'filtered taxes'
df = filtered_tax.merge(gas_prod_v2, on="Year")
# Dropping 2019 due to incomplete data; getting a prediction for 2019 up to 2023
df = df[df['Year'] <= 2018]
df

,Year,US Tax (% on Goods & Services),World Tax (% on Goods & Services),US Inflation (Annual %),World Inflation (Annual %),US GDP (%),World GDP (%),World Population,Canada,Mexico,...,India,Indonesia,Myanmar,Pakistan,Other Asia Pacific,Total Asia Pacific,World Exajoules,of which: OECD,Non-OECD,European Union #
0,1970,0.000000,0.000000,5.838255,0.000000,21.414736,26.911121,3682911039,1.94,0.40,...,0.02,0.05,^,0.10,0.20,0.54,35.14,26.04,9.10,3.74
1,1971,0.000000,0.000000,4.292767,0.000000,21.919818,26.526697,3760509002,2.13,0.39,...,0.02,0.05,^,0.10,0.22,0.64,37.58,27.62,9.97,4.57
2,1972,7.143859,0.000000,3.272278,0.000000,22.580622,26.164536,3836892580,2.40,0.41,...,0.03,0.04,^,0.10,0.24,0.75,39.44,28.80,10.64,5.51
3,1973,6.579487,0.000000,6.177760,0.000000,23.331809,27.054172,3912347640,2.57,0.44,...,0.03,0.06,^,0.11,0.25,0.93,41.33,29.66,11.67,6.19
4,1974,5.990202,0.000000,11.054805,0.000000,22.694942,27.896165,3988478324,2.52,0.47,...,0.03,0.06,0.01,0.12,0.26,1.13,42.20,29.39,12.81,6.88
5,1975,5.626993,0.000000,9.143147,0.000000,20.277476,25.909123,4062864562,2.57,0.47,...,0.04,0.09,0.01,0.14,0.25,1.28,42.31,28.17,14.14,7.16
6,1976,5.410532,0.000000,5.744813,0.000000,22.038398,26.450586,4135418002,2.60,0.46,...,0.05,0.09,0.01,0.14,0.28,1.47,44.13,28.47,15.66,7.61
7,1977,4.678032,0.000000,6.501684,0.000000,23.526270,26.629795,4207766711,2.73,0.49,...,0.05,0.21,0.01,0.15,0.32,1.80,45.99,28.93,17.06,7.73
8,1978,4.357011,0.000000,7.630964,0.000000,24.831785,27.184231,4281312782,2.63,0.62,...,0.06,0.40,0.01,0.16,0.30,2.14,47.55,29.08,18.47,7.50
9,1979,3.794360,0.000000,11.254471,0.000000,25.111120,27.392791,4356746035,2.77,0.75,...,0.07,0.58,0.01,0.18,0.27,2.43,50.76,30.23,20.52,7.54


### Scaling and training the data using SKLearn Lasso and StandardScaler

In [3]:
#  # LASSO model
# # Note: Use an alpha of .01 when creating the model for this activity
# from sklearn.linear_model import Lasso

# ### BEGIN SOLUTION
# lasso = Lasso(alpha=.01).fit(X_train_scaled, y_train_scaled)

# predictions = lasso.predict(X_test_scaled)

# MSE = mean_squared_error(y_test_scaled, predictions)
# r2 = lasso.score(X_test_scaled, y_test_scaled)
# ### END SOLUTION

# print(f"MSE: {MSE}, R2: {r2}")

In [4]:
# Create the model
model = Lasso(alpha=.01)

### One-Step-Ahead Forecast

In [5]:
# Using 2000 - 2009 data to run historical prediction from 2001 - 2010
predict_0110=[]

for year in range(10):
    i = 31 + year

    # Does not need .value.reshape(-1, 1) as there's dimension now with 2+ features
    hist_X = df[["World Population",
                 "World GDP (%)",
                 "World Inflation (Annual %)",
                 "World Tax (% on Goods & Services)"]]
    hist_y = df["World Exajoules"].values.reshape(-1, 1)
    X_scaler = StandardScaler().fit(hist_X)
    y_scaler = StandardScaler().fit(hist_y)
    X_train_scaled = X_scaler.transform(hist_X)
    y_train_scaled = y_scaler.transform(hist_y)
    X_train_scaled = pd.DataFrame(X_train_scaled)
    
    # Fitting 1980-2000 therefore, i-21 on the first iteration and so on up to 10x
    lasso = model.fit(X_train_scaled.iloc[(i-21):i], y_train_scaled[(i-21):i])
    
    # Changed reshape to (1, -1)
    gas_predict = lasso.predict(X_train_scaled.iloc[i-1].values.reshape(1, -1))
    predict_0110.append(gas_predict.flatten()[0])
    
# Invert predict_0110 so it's not scaled for later comparison
inverse_predict_0110 = y_scaler.inverse_transform(predict_0110)

print(inverse_predict_0110)

[ 84.22071381  86.27672611  88.39214669  90.79808905  93.15247727
  95.38828569  98.30215337 101.79200536 104.92339188 103.25506153]


### Historical Prediction MSE and R-Square

In [6]:
# Use our model to make predictions
predicted = lasso.predict(X_train_scaled)

inverse_predicted = y_scaler.inverse_transform(predicted)

hist_mse = mean_squared_error(y_train_scaled, predicted)
hist_r2 = lasso.score(X_train_scaled, y_train_scaled)

print(f"-----------------Historical-----------------")
print(f"Mean Squared Error (MSE): {hist_mse}")
print(f"R-squared (R2): {hist_r2}")

-----------------Historical-----------------
Mean Squared Error (MSE): 0.026202903558623106
R-squared (R2): 0.973797096441377


### Historical Predictions

In [7]:
historical_comps_0110 = df.loc[df["Year"].between(2001, 2010), ["Year", "World Exajoules"]]
historical_comps_0110["Prediction"] = inverse_predict_0110
historical_comps_0110["Difference"] = historical_comps_0110["Prediction"] - historical_comps_0110["World Exajoules"]
historical_comps_0110["% Difference"] = ((historical_comps_0110["Prediction"] - historical_comps_0110["World Exajoules"])/historical_comps_0110["World Exajoules"]) * 100
historical_comps_0110

,Year,World Exajoules,Prediction,Difference,% Difference
31,2001,88.51,84.220714,-4.289286,-4.846103
32,2002,90.54,86.276726,-4.263274,-4.708719
33,2003,93.79,88.392147,-5.397853,-5.755255
34,2004,96.86,90.798089,-6.061911,-6.258426
35,2005,99.16,93.152477,-6.007523,-6.058413
36,2006,102.53,95.388286,-7.141714,-6.965487
37,2007,105.16,98.302153,-6.857847,-6.521345
38,2008,109.06,101.792005,-7.267995,-6.664217
39,2009,105.66,104.923392,-0.736608,-0.697149
40,2010,113.26,103.255062,-10.004938,-8.833603


### Exporting Historical Predictions to CSV
* Find it in the 'clean_data' folder within 'data'

In [8]:
# Saving historical predictions to a csv file
historical_comps_0110.to_csv('../data/clean_data/gas_outputs/FINAL_OUTPUTS/GasProduction_Historical_LassoModel_2001_2010.csv', index=False)

### Calculating the moving averages for the features from 2019-2023

In [9]:
# Narrow down data frame to the specific year range of 2010 - 2018
MA = df.loc[df["Year"].between(2010, 2018), ["Year",
                                             "World Population",
                                             "World Exajoules",
                                             "World GDP (%)",
                                             "World Inflation (Annual %)",
                                             "World Tax (% on Goods & Services)"]]
MA

,Year,World Population,World Exajoules,World GDP (%),World Inflation (Annual %),World Tax (% on Goods & Services)
40,2010,6921871614,113.26,24.207113,3.326345,31.875890
41,2011,7002860604,117.07,24.547417,4.839403,33.264196
42,2012,7085763408,119.48,24.404915,3.707818,33.271756
43,2013,7169640142,120.88,24.310278,2.605818,32.787076
44,2014,7254228377,123.33,24.470283,2.346269,33.191709
45,2015,7338964960,126.02,24.297531,1.393330,33.724915
46,2016,7424282488,127.45,23.913640,1.486007,34.248831
47,2017,7509065705,132.21,24.222791,2.233522,33.333664
48,2018,7591932907,138.87,24.382773,2.458142,34.011405


In [10]:
# Iterate 5 times through 5 years (2019-2023) of the features' moving averages
for i in range(5):
    starting_index = 4 + i
    year_inc = 2018 + i
    new_year = year_inc + 1

    pop_mean = MA['World Population'].iloc[starting_index:starting_index+5].mean()
    gdp_mean = MA['World GDP (%)'].iloc[starting_index:starting_index+5].mean()
    inf_mean = MA['World Inflation (Annual %)'].iloc[starting_index:starting_index+5].mean()
    tax_mean = MA['World Tax (% on Goods & Services)'].iloc[starting_index:starting_index+5].mean()

    df = pd.DataFrame({"Year":[new_year],
                       "World Population": [pop_mean],
                       "World Exajoules": 0,
                       "World GDP (%)": [gdp_mean],
                       "World Inflation (Annual %)": [inf_mean],
                       "World Tax (% on Goods & Services)": [tax_mean]})
    
    MA = MA.append(df, ignore_index=True)
    del df
MA

,Year,World Population,World Exajoules,World GDP (%),World Inflation (Annual %),World Tax (% on Goods & Services)
0,2010,6.921872e+09,113.26,24.207113,3.326345,31.875890
1,2011,7.002861e+09,117.07,24.547417,4.839403,33.264196
2,2012,7.085763e+09,119.48,24.404915,3.707818,33.271756
3,2013,7.169640e+09,120.88,24.310278,2.605818,32.787076
4,2014,7.254228e+09,123.33,24.470283,2.346269,33.191709
5,2015,7.338965e+09,126.02,24.297531,1.393330,33.724915
6,2016,7.424282e+09,127.45,23.913640,1.486007,34.248831
7,2017,7.509066e+09,132.21,24.222791,2.233522,33.333664
8,2018,7.591933e+09,138.87,24.382773,2.458142,34.011405
9,2019,7.423695e+09,0.00,24.257404,1.983454,33.702105


### Multi-Step Forecast

In [11]:
future_predict = []

# Updates: relocated some sections of code
future_X = MA[["World Population",
                 "World GDP (%)",
                 "World Inflation (Annual %)",
                 "World Tax (% on Goods & Services)"]]
future_y = MA["World Exajoules"].values.reshape(-1, 1)
X_scaler = StandardScaler().fit(future_X)
y_scaler = StandardScaler().fit(future_y[0:9])
X_test_scaled = X_scaler.transform(future_X)
y_test_scaled = y_scaler.transform(future_y[0:9])
X_test_scaled_df = pd.DataFrame(X_test_scaled)
y_test_scaled_df = pd.DataFrame(y_test_scaled)

for year in range(5):
    i = 9 + year

    lasso = model.fit(X_test_scaled_df.iloc[year:i], y_test_scaled_df.iloc[year:i+1])
    gas_predict_2 = lasso.predict(X_test_scaled_df.iloc[i-1].values.reshape(1, -1))
    df2 = pd.DataFrame(pd.Series(gas_predict_2.flatten()[0]))
    future_predict.append(gas_predict_2.flatten()[0])
    
    y_test_scaled_df = y_test_scaled_df.append(df2, ignore_index=True)
    del df2

# Invert future_predict so it's not scaled for later comoparison
inverse_future_predict = y_scaler.inverse_transform(future_predict)

print(inverse_future_predict)

[136.84941498 130.87061295 131.80401885 132.5915376  133.30492108]


### Predictions for Future 'World Exajoules' according to the ML

In [12]:
future_comps_19_23 = MA.loc[MA["Year"].between(2019, 2023), ["Year"]]
future_comps_19_23["Prediction"] = inverse_future_predict
future_comps_19_23

,Year,Prediction
9,2019,136.849415
10,2020,130.870613
11,2021,131.804019
12,2022,132.591538
13,2023,133.304921


### Exporting Future Predictions to CSV
* Find it in the 'clean_data' folder within 'data'

In [13]:
# Saving future predictions to a csv file
future_comps_19_23.to_csv('../data/clean_data/gas_outputs/FINAL_OUTPUTS/GasProduction_Future_LassoModel_2019_2023.csv', index=False)

### Building a Summary Table for MSE and R-Square

In [14]:
# Building a summary table for the historical MSE and R-Square
stats = [["Lasso",
          hist_mse,
          hist_r2]]
score_summary = pd.DataFrame(stats, columns = ["Model",
                                               "Historical MSE",
                                               "Historical R-Square"])
score_summary

,Model,Historical MSE,Historical R-Square
0,Lasso,0.026203,0.973797


### Exporting Summary Table for MSE and R-Square
* Find it in the 'gas_outputs' folder within 'clean_data'

In [15]:
score_summary.to_csv('../data/clean_data/gas_outputs/FINAL_OUTPUTS/GasProduction_MSE_R2_Lasso_Table.csv', index=False)